# ***Car Prediction Model using VGG***
This code aims to create and train model for car prediction. Here the transfer learning technique is used create the model.


**Libraries**

In [ ]:
#Imporatant Libraries for working with arrays.
import numpy as np

#Important Libraries for visulization of the results.
import matplotlib.pyplot as plt

#Important Libraries for training and testing of the model.
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
#configuring the gpu 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
tf.version
assert tf.test.is_built_with_cuda()
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)


**Model Generator**

In [ ]:
def Model_Generator(IMAGE_SIZE):

    vgg =VGG16(input_shape= [227,227, 3], weights='imagenet', include_top=False)

    for layer in vgg.layers[:17]:
        layer.trainable = False

    folders = glob(r'D:\car196\newtrain1\*')

    conv_output = vgg.get_layer("input_1").output  
    x = layers.Conv2D(64, (3, 3),
                      activation='selu',
                      padding='same',
                      name='block1_conv1')(conv_output)
    x = layers.Conv2D(64, (3, 3),
                      activation='selu',
                      padding='same',
                      name='block1_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    x = layers.Conv2D(128, (3, 3),
                      activation='selu',
                      padding='same',
                      name='block2_conv1')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='selu',
                      padding='same',
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block2_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)


    x = layers.Conv2D(256, (3, 3),
                      activation='selu',
                      padding='same',
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block3_conv1')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='selu',
                      padding='same',
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block3_conv2')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='selu',
                      padding='same',
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block3_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)


    x = layers.Conv2D(512, (3, 3),
                      activation='selu',
                      padding='same',
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block4_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='selu',
                      padding='same',
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block4_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='selu',
                      padding='same',
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block4_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='selu',
                      padding='same',
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block5_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='selu',
                      padding='same',
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block5_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='selu',
                      padding='same',  
                      kernel_regularizer =tf.keras.regularizers.l2(0.01),
                      name='block5_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    model = Model(inputs=vgg.input, outputs=x)
    model.load_weights(vgg)
    x = Flatten()(model.output)
    prediction = Dense(len(folders), activation='softmax')(x)
    model = Model(inputs=vgg.input, outputs=prediction)

  #Printting Information about the model.
    print(model.summary())
    return model

**Optimizer generator**

In [ ]:
def Optimizer_Generator():
  # Set the parameters value for optimizer
  adam=tf.keras.optimizers.Adam(
          learning_rate=1e-3,
          beta_1=0.9,
          beta_2=0.999,
          epsilon=1e-07,
          decay = 0.0
  )
  return adam

**Model training**

In [ ]:
def Model_training(training_set, test_set, callback):
  results = model.fit(
                        training_set,
                        validation_data=test_set,
                        epochs=20,
                        steps_per_epoch=len(training_set),
                        validation_steps=len(test_set),
                        callbacks = [callback]
                      )
  return results

In [ ]:
#Dimension of input image.  
Image_Dimension = [227,227]

#Generation of the model
model = Model_Generator(Image_Dimension)

#Generation of the optimizer
adam = Optimizer_Generator()

#Compile model.
model.compile(
  loss='categorical_crossentropy',
  optimizer=adam,
  metrics=['accuracy']
)

#Generating flow from the directories for training and testing dataset.

#Genrating flow of images from training dataset.
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range = 30,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('D:/car196/car100/training',
                                                 target_size = (227, 227),
                                                 batch_size = 256,
                                                 class_mode = 'categorical',
                                                 shuffle=True)

#Genrating flow of images from testing dataset.
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_set = test_datagen.flow_from_directory('D:/car196/car100/testing',
                                            target_size = (227, 227),
                                            batch_size = 256,
                                            class_mode = 'categorical',
                                            shuffle=True)

#Set Callback options here.
checkpoint_filepath = 'D:/car196/car100/weights.{epoch:02d}val_acc{val_accuracy:3f}.hdf5'
callback = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4,mode = 'auto', restore_best_weights=False),
            tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.1, patience=2, mode="auto",min_delta=0.0001,min_lr=0),
            tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True),
           ]


results = Model_training(training_set, test_set, callback)

#Visualizing the results.

#Plot for epoch versus loss.
plt.plot(results.history['loss'], label='train loss')
plt.plot(results.history['val_loss'], label='val loss')
plt.legend()
plt.show()

#Plot for epoch versus accuracy.
plt.plot(results.history['accuracy'], label='train acc')
plt.plot(results.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
 
#Save model 
model.save('D:/car196/car100/New_model_100_classes_VGG.h5')